In [3]:
from langchain_community.chat_models import ChatOllama 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from utils import debug_print
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
import ollama
import re

def analyze_persuasion(message: str) -> int:
    """
    Analyzes the persuasion power of a message and returns a score between -10 and 10.
    """
    prompt = f"""Evaluate the argument's persuasive power using these criteria:
1. Emotional impact (-10 to 10)
2. Logical consistency (-10 to 10)
3. Evidence quality (-10 to 10)
4. Rhetorical effectiveness (-10 to 10)
5. Overall tone (-10 to 10)

Calculate the average score and return ONLY ONE INTEGER between -10 and 10.
Negative scores indicate counterproductive arguments.

Argument: {message}
Final Score:"""
    
    response = ollama.generate(
        model='mistral',
        prompt=prompt,
        options={'temperature': 0.3}
    )

    match = re.search(r'-?\b(10|[0-9])\b', response['response'])
    
    if match:
        score = int(match.group())
        return max(-10, min(10, score))
    return 0

user_input = """2. **Gun Violence Epidemic**: The United States experiences a high rate of
gun-related deaths and injuries compared to other developed countries.
Limiting access to firearms could help mitigate this epidemic and reduce
the number of lives lost."""
persuasion_score = analyze_persuasion(user_input)
print(f"Persuasion Score: {persuasion_score}")

Persuasion Score: 4


In [5]:
def get_argument_by_number(file_path, argument_number):
    with open(file_path, "r") as file:
        content = file.read()
    arguments = content.split("\n\n")
    if argument_number < 1 or argument_number > len(arguments):
        return "Invalid argument number. Please provide a number between 1 and 25."
    return arguments[argument_number - 1].strip()

statement = "People should have a right to keep and bear arms."

In [9]:
negative_scores = [[3, 4, 1, 1, 4, 4, 5, 5, 4, 6, 4, 3, 4, 3, 3, 3, 5, 3, 6, 3, 4, 3, 1, -10, 1], [], []]
for k in range(1, 3):
    for i in range(1, 26):
        arg = get_argument_by_number("con_arguments.txt", i)
        persuasion_score = analyze_persuasion(arg)
        print(f"{k} Persuasion score of argument {i}: {persuasion_score}")
        negative_scores[k].append(persuasion_score)

1 Persuasion score of argument 1: 3
1 Persuasion score of argument 2: 4
1 Persuasion score of argument 3: 1
1 Persuasion score of argument 4: -10
1 Persuasion score of argument 5: 4
1 Persuasion score of argument 6: 5
1 Persuasion score of argument 7: 4
1 Persuasion score of argument 8: 6
1 Persuasion score of argument 9: 5
1 Persuasion score of argument 10: 6
1 Persuasion score of argument 11: 3
1 Persuasion score of argument 12: 3
1 Persuasion score of argument 13: 4
1 Persuasion score of argument 14: 3
1 Persuasion score of argument 15: 3
1 Persuasion score of argument 16: 3
1 Persuasion score of argument 17: 5
1 Persuasion score of argument 18: 3
1 Persuasion score of argument 19: 6
1 Persuasion score of argument 20: 3
1 Persuasion score of argument 21: 4
1 Persuasion score of argument 22: 3
1 Persuasion score of argument 23: 6
1 Persuasion score of argument 24: 5
1 Persuasion score of argument 25: 3
2 Persuasion score of argument 1: 3
2 Persuasion score of argument 2: 4
2 Persuasi

In [10]:
negative_scores

[[3, 4, 1, 1, 4, 4, 5, 5, 4, 6, 4, 3, 4, 3, 3, 3, 5, 3, 6, 3, 4, 3, 1, -10, 1],
 [3, 4, 1, -10, 4, 5, 4, 6, 5, 6, 3, 3, 4, 3, 3, 3, 5, 3, 6, 3, 4, 3, 6, 5, 3],
 [3, 4, -10, 1, 4, 5, 5, 5, 4, 6, 3, 4, 4, 3, 3, 3, 5, 4, 6, 3, 4, 3, 4, 6, 3]]